In [1]:
import polars as pl

In [2]:
pollutant = pl.read_csv("../../data/raw/pollutant_data.csv")
measurement = pl.read_csv("../../data/raw/measurement_data.csv")
instrument = pl.read_csv("../../data/raw/instrument_data.csv")

In [3]:
questions = {
    "target": {
    }
}

In [4]:
measurement = measurement.join(instrument, on=["Measurement date", "Station code"])
measurement = measurement.filter(pl.col("Instrument status") == 0)

## Question 1

In [5]:
measurement.head()

Measurement date,Station code,Latitude,Longitude,SO2,NO2,O3,CO,PM10,PM2.5,Item code,Average value,Instrument status
str,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64
"""2021-01-01 00:00:00""",204,37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0,0,0.004,0
"""2021-01-01 00:00:00""",204,37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0,2,0.059,0
"""2021-01-01 00:00:00""",204,37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0,4,1.2,0
"""2021-01-01 00:00:00""",204,37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0,5,0.002,0
"""2021-01-01 00:00:00""",204,37.572016,127.005008,0.004,0.059,0.002,1.2,73.0,57.0,7,73.0,0


In [6]:
measurement = (
    measurement
    .with_columns(
        pl.col("Measurement date").str.to_datetime()
    )
    .with_columns(
        pl.col("Measurement date").dt.day().alias("day"),
        pl.col("Measurement date").dt.month().alias("month"),
        pl.col("Measurement date").dt.year().alias("year"),
    )
)

In [7]:
q1 = (
    measurement
    .group_by(["year", "month", "day"])
    .agg(pl.mean("SO2"))
    .select("SO2")
    .mean()
)
response_q1 = q1["SO2"].to_list()[0]
questions["target"]["Q1"] = round(response_q1, 5)

## Question 2

In [8]:
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 1
    elif month in [3, 4, 5]:
        return 2
    elif month in [6, 7, 8]:
        return 3
    else:
        return 4

In [9]:
q2 = (
    measurement
    .filter(pl.col("Station code") == 209)
    .with_columns(
        pl.col("Measurement date").map_elements(get_season, return_dtype=pl.Int64).alias("season")
    )
    .group_by("season")
    .agg(pl.col("CO").mean())
)
season_1 = q2.filter(pl.col("season") == 1)["CO"].to_list()[0]
season_2 = q2.filter(pl.col("season") == 2)["CO"].to_list()[0]
season_3 = q2.filter(pl.col("season") == 3)["CO"].to_list()[0]
season_4 = q2.filter(pl.col("season") == 4)["CO"].to_list()[0]
questions["target"]["Q2"] = {
    "1" : round(season_1, 5),
    "2" : round(season_2, 5),
    "3" : round(season_3, 5),
    "4" : round(season_4, 5)
}

## Question 3

In [10]:
measurement = (
    measurement
   .with_columns(
        pl.col("Measurement date").dt.hour().alias("hour")
    )
)

std_per_hour = (
    measurement
    .group_by("hour")
    .agg(pl.col("O3").std().alias("std_O3"))
    .sort("std_O3", descending=True)
    .head(1)
)

response_3 = std_per_hour["hour"].to_list()[0]
questions["target"]["Q3"] = round(response_3, 5)

## Write the answers

In [11]:
import json
with open("../../predictions/questions.json", "w") as file:
    json.dump(questions, file)